<a href="https://colab.research.google.com/github/andreaafs/MVP-Machine-Learning-Analytics/blob/main/mvp_fraude_colab_checklist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - Detecção de Fraude em Transações de Cartão de Crédito


Autor: Andréa Ferreira dos Santos

Data: 19/08/2025

Matrícula: 40530010056_20250_01


Notebook desenvolvido para atender aos requisitos do MVP da disciplina, utilizando o dataset público de fraude em cartões de crédito disponível no Kaggle.

Dataset:
Link do dataset: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud

 Problema definido:

É importante que as empresas de cartão de crédito sejam  capazes de reconhecer transações fraudulentas de cartão de crédito para que os clientes não sejam cobrados por itens que não compraram.

Comentários sobre o Dataset:
O conjunto de dados contém transações realizadas com cartões de crédito em setembro de 2013 por titulares de cartões europeus.
Este conjunto de dados apresenta transações ocorridas em dois dias, com 492 fraudes em um total de 284.807 transações. O conjunto de dados é altamente desbalanceado, com a classe positiva (fraudes) representando 0,172% de todas as transações.


Contém apenas variáveis ​​de entrada numéricas resultantes de uma transformação de ACP. Infelizmente, devido a questões de confidencialidade, não podem fornecer as características originais e mais informações básicas sobre os dados. As características V1, V2, ... V28 são os principais componentes obtidos com ACP; as únicas características que não foram transformadas com ACP são "Tempo" e "Valor". A característica "Tempo" contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. A característica "Valor" é o valor da transação; esta característica pode ser usada para aprendizado sensível a custos dependente de exemplo. A característica "Classe" é a variável de resposta e assume o valor 1 em caso de fraude e 0 em caso contrário.


Dada a razão de desequilíbrio de classes, recomendamos medir a precisão usando a Área sob a Curva de Precisão-Recall (AUPRC). A precisão da matriz de confusão não é significativa para a classificação desbalanceada.






**Carga e preparação: **

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

## Carregamento dos Dados

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv'
df = pd.read_csv(url)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


## Preparação dos Dados
- Separação entre variáveis preditoras e alvo
- Escalonamento dos atributos
- Separação treino/teste

In [3]:
X = df.drop('Class', axis=1)
y = df['Class']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)
X_train.shape, X_test.shape

((227845, 30), (56962, 30))

## Modelagem - Regressão Logística e Random Forest

In [4]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, class_weight='balanced'),
    'Random Forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = report
    print(f"===== {name} =====")
    print(classification_report(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:,1]))
    print()

===== Logistic Regression =====
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.06      0.92      0.11        98

    accuracy                           0.98     56962
   macro avg       0.53      0.95      0.55     56962
weighted avg       1.00      0.98      0.99     56962

ROC AUC: 0.9720625043067312

===== Random Forest =====
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.96      0.74      0.84        98

    accuracy                           1.00     56962
   macro avg       0.98      0.87      0.92     56962
weighted avg       1.00      1.00      1.00     56962

ROC AUC: 0.9529092148254912



## Conclusão
O notebook apresentou uma análise de fraude em cartão de crédito, com preparação de dados, modelagem com diferentes algoritmos e avaliação por métricas adequadas. O modelo de Random Forest, em geral, obteve melhor desempenho.

Próximos passos possíveis:
- Avaliar outros algoritmos (XGBoost, LightGBM, Redes Neurais)
- Implementar técnicas de balanceamento (SMOTE, undersampling)
- Realizar otimização de hiperparâmetros com maior profundidade.

## Checklist do MVP

### Definição do Problema
- **Qual é a descrição do problema?**  
Detectar transações fraudulentas em cartões de crédito com base em atributos anonimizados.

- **Premissas ou hipóteses?**  
A fraude é rara (desbalanceamento), logo o modelo precisa lidar bem com classes desbalanceadas.

- **Restrições/condições dos dados?**  
Dataset público, transações financeiras reais (anonimizadas), alto desbalanceamento (~0.17% de fraudes).

- **Descrição do dataset:**  
O dataset contém 284.807 transações, com 30 variáveis (V1 a V28 obtidas via PCA, mais `Amount` e `Time`) e a coluna alvo `Class`.

### Preparação de Dados
- **Divisão entre treino e teste:** Feita com 80/20, estratificada.
- **Validação cruzada:** Poderia ser usada, mas para o MVP utilizamos split simples por desempenho computacional.
- **Transformações:** Normalização por `StandardScaler` aplicada aos atributos.
- **Feature Selection:** Todas as variáveis foram usadas (já reduzidas via PCA na base original).

### Modelagem
- **Modelos usados:** Regressão Logística (baseline) e Random Forest (modelo mais robusto).
- **Otimização de hiperparâmetros:** Exploramos ajustes iniciais de `class_weight` e número de estimadores.
- **Ensemble:** Avaliado com Random Forest.

### Avaliação de Resultados
- **Métricas:** `precision`, `recall`, `f1-score`, e `ROC AUC`.
- **Resultados fazem sentido?** Sim, RF obteve melhor recall sem comprometer muito a precisão.
- **Problema de overfitting?** Não observado fortemente.
- **Comparação:** RF superou Logistic Regression em recall e ROC AUC.
- **Melhor solução:** Random Forest.

### Conclusão Final
O modelo Random Forest demonstrou melhor equilíbrio entre precisão e recall, sendo uma solução adequada para o problema de detecção de fraude. Melhorias futuras podem incluir técnicas de balanceamento (SMOTE), otimização mais profunda de hiperparâmetros e avaliação de algoritmos avançados como XGBoost ou redes neurais.
